# Hate Speech Detection

In [67]:
import pandas as pd
import string
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [68]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer as VS

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import seaborn as sns
from textstat.textstat import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [37]:
## Reading the dataset

In [70]:
data = pd.read_csv("Hatespeech.csv")
data

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [71]:
data['text length'] = data['tweet'].apply(len)

In [72]:
# collecting only the tweets from the csv file into a variable name tweet
tweet=data.tweet

### Preprocessing of the major 'x' variable

In [73]:
## 1. Removal of punctuation and capitlization
## 2. Tokenizing
## 3. Removal of stopwords
## 4. Stemming

stopwords = nltk.corpus.stopwords.words("english")

#extending the stopwords to include other words used in twitter such as retweet(rt) etc.
other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)
stemmer = PorterStemmer()

def preprocess(tweet):  
    
    # removal of extra spaces
    regex_pat = re.compile(r'\s+')
    tweet_space = tweet.str.replace(regex_pat, ' ')

    # removal of @name[mention]
    regex_pat = re.compile(r'@[\w\-]+')
    tweet_name = tweet_space.str.replace(regex_pat, '')

    # removal of links[https://abc.com]
    giant_url_regex =  re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweets = tweet_name.str.replace(giant_url_regex, '')
    
    # removal of punctuations and numbers
    punc_remove = tweets.str.replace("[^a-zA-Z]", " ")
    
    # remove whitespace with a single space
    newtweet=punc_remove.str.replace(r'\s+', ' ')
    
    # remove leading and trailing whitespace
    newtweet=newtweet.str.replace(r'^\s+|\s+?$','')
    
    # replace normal numbers with numbr
    newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr')
    
    # removal of capitalization
    tweet_lower = newtweet.str.lower()
    
    # tokenizing
    tokenized_tweet = tweet_lower.apply(lambda x: x.split())
    
    # removal of stopwords
    tokenized_tweet=  tokenized_tweet.apply(lambda x: [item for item in x if item not in stopwords])
    
    # stemming of the tweets
    tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 
    
    for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
        tweets_p= tokenized_tweet
    
    return tweets_p

processed_tweets = preprocess(tweet)   

data['processed_tweets'] = processed_tweets
print(data['tweet'])
print(data['processed_tweets'])

0        !!! RT @mayasolovely: As a woman you shouldn't...
1        !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2        !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3        !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4        !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
                               ...                        
24778    you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779    you've gone and broke the wrong heart baby, an...
24780    young buck wanna eat!!.. dat nigguh like I ain...
24781                youu got wild bitches tellin you lies
24782    ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...
Name: tweet, Length: 24783, dtype: object
0        woman complain clean hous amp man alway take t...
1        boy dat cold tyga dwn bad cuffin dat hoe st place
2               dawg ever fuck bitch start cri confus shit
3                                         look like tranni
4           shit hear might true might faker bitch told ya
              

### Feature Engineering

In [ ]:
### TFIDF

In [136]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_df=0.75, min_df=5, max_features=10000) 
tfidf = tfidf_vectorizer.fit_transform(data['processed_tweets'] )
print(tfidf)

  (0, 5549)	0.4309517500614609
  (0, 5823)	0.21292360474103067
  (0, 5536)	0.2694725323170756
  (0, 103)	0.2848504306418305
  (0, 3639)	0.2500076439754137
  (0, 124)	0.22951771468335722
  (0, 2896)	0.32473278046308335
  (0, 1189)	0.37180372772740344
  (0, 1266)	0.37805333846740247
  (0, 6262)	0.33673876130852654
  (1, 1455)	0.35408569797294004
  (1, 4385)	0.29402989193760287
  (1, 5340)	0.3008029767463628
  (1, 2717)	0.12204320836205955
  (1, 1388)	0.37776257623201304
  (1, 321)	0.20828940991978914
  (1, 5944)	0.35642794983844883
  (1, 1221)	0.3008029767463628
  (1, 1453)	0.47084617485060926
  (1, 892)	0.2397588136195512
  (2, 723)	0.4025857370799935
  (2, 2121)	0.26875711871483904
  (2, 1802)	0.4334907503536408
  (2, 5054)	0.19413584053699529
  (2, 1279)	0.366125652299236
  :	:
  (24780, 960)	0.343011332843257
  (24780, 63)	0.37409643142606824
  (24780, 4095)	0.26604418498734955
  (24780, 1544)	0.2613777761879398
  (24780, 2187)	0.2280990649157048
  (24780, 1713)	0.21384505532921141
 

In [137]:
tfidf_a = tfidf.toarray()
tfidf_a.shape

(24783, 6441)

In [ ]:
# Sentiment Analysis, using polarity scores as features....

In [83]:
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sasha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Sasha\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [87]:
sentiment_analyzer = VS()

# Method to tag tags
def count_tags(tweet_c):  
    
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', tweet_c)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

# Method to capture and analyse sentiment
def sentiment_analysis(tweet):   
    sentiment = sentiment_analyzer.polarity_scores(tweet)    
    twitter_objs = count_tags(tweet)
    features = [sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],twitter_objs[0], twitter_objs[1],
                twitter_objs[2]]
    #features = pandas.DataFrame(features)
    return features

# Method to store the sentimental analysis features
def sentiment_analysis_array(tweets):
    features=[]
    for t in tweets:
        features.append(sentiment_analysis(t))
    return np.array(features)

final_features = sentiment_analysis_array(tweet)
final_features

array([[0.   , 0.12 , 0.88 , ..., 0.   , 1.   , 0.   ],
       [0.237, 0.   , 0.763, ..., 0.   , 1.   , 0.   ],
       [0.538, 0.   , 0.462, ..., 0.   , 2.   , 0.   ],
       ...,
       [0.   , 0.219, 0.781, ..., 0.   , 0.   , 0.   ],
       [0.573, 0.   , 0.427, ..., 0.   , 0.   , 0.   ],
       [0.   , 0.218, 0.782, ..., 1.   , 0.   , 0.   ]])

In [86]:
# Converting the array to Dataframe

new_features = pd.DataFrame({'-ve':final_features[:,0],'+ve':final_features[:,1],'Neutral':final_features[:,2],'Compound':final_features[:,3],
                            'url_tag':final_features[:,4],'mention_tag':final_features[:,5],'hash_tag':final_features[:,6]})
new_features

,-ve,+ve,Neutral,Compound,url_tag,mention_tag,hash_tag
0,0.000,0.120,0.880,0.4563,0.0,1.0,0.0
1,0.237,0.000,0.763,-0.6876,0.0,1.0,0.0
2,0.538,0.000,0.462,-0.9550,0.0,2.0,0.0
3,0.000,0.344,0.656,0.5673,0.0,2.0,0.0
4,0.249,0.081,0.669,-0.7762,0.0,1.0,1.0
...,...,...,...,...,...,...,...
24778,0.000,0.000,1.000,0.0000,0.0,3.0,3.0
24779,0.454,0.000,0.546,-0.8074,0.0,0.0,0.0
24780,0.000,0.219,0.781,0.4738,0.0,0.0,0.0
24781,0.573,0.000,0.427,-0.7717,0.0,0.0,0.0


In [ ]:
### Doc2Vec

In [88]:
# Vectorization using Doc2Vec
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [89]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data["processed_tweets"].apply(lambda x: x.split(" ")))]

# Usage of Doc2Vec
model = Doc2Vec(documents,vector_size=5, window=2, min_count=1, workers=4)

In [90]:
# transform each document into a vector data
doc2vec_df = data["processed_tweets"].apply(lambda x: model.infer_vector(x.split(" "))).apply(pd.Series)
doc2vec_df.columns = ["doc2vec_vector_" + str(x) for x in doc2vec_df.columns]
doc2vec_df

,doc2vec_vector_0,doc2vec_vector_1,doc2vec_vector_2,doc2vec_vector_3,doc2vec_vector_4
0,-0.067676,0.067442,-0.053648,-0.068619,0.010614
1,0.060424,-0.067749,0.282897,-0.011470,0.035286
2,0.080564,0.117252,0.038254,-0.036045,0.073248
3,-0.050044,0.162034,0.014664,-0.056884,-0.043545
4,0.019733,0.018483,0.050602,-0.074799,-0.161415
...,...,...,...,...,...
24778,0.221209,0.134886,0.075971,-0.356613,-0.020821
24779,-0.068515,0.021608,0.110659,-0.015455,-0.005765
24780,-0.058928,0.157185,0.214449,0.096644,-0.160514
24781,-0.035652,0.131133,0.142956,-0.067903,-0.066157


In [ ]:
### Clubing of tf-idf scores, sentiment scores and doc2vec columns

In [116]:
modelling_features = np.concatenate([tfidf_a,final_features,doc2vec_df],axis=1) #To add tfidf_a
modelling_features.shape

(24783, 6453)

# Running the models Using TFIDF with additional features from sentiment analysis and doc2vec

In [49]:
### Independent variable

In [117]:
X = pd.DataFrame(modelling_features)
X.shape

(24783, 6453)

In [119]:
X

,0,1,2,3,4,5,6,7,8,9,...,6443,6444,6445,6446,6447,6448,6449,6450,6451,6452
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.880,0.4563,0.0,1.0,0.0,-0.067676,0.067442,-0.053648,-0.068619,0.010614
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.763,-0.6876,0.0,1.0,0.0,0.060424,-0.067749,0.282897,-0.011470,0.035286
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.462,-0.9550,0.0,2.0,0.0,0.080564,0.117252,0.038254,-0.036045,0.073248
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.656,0.5673,0.0,2.0,0.0,-0.050044,0.162034,0.014664,-0.056884,-0.043545
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.669,-0.7762,0.0,1.0,1.0,0.019733,0.018483,0.050602,-0.074799,-0.161415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000,0.0000,0.0,3.0,3.0,0.221209,0.134886,0.075971,-0.356613,-0.020821
24779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.546,-0.8074,0.0,0.0,0.0,-0.068515,0.021608,0.110659,-0.015455,-0.005765
24780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.781,0.4738,0.0,0.0,0.0,-0.058928,0.157185,0.214449,0.096644,-0.160514
24781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.427,-0.7717,0.0,0.0,0.0,-0.035652,0.131133,0.142956,-0.067903,-0.066157


In [51]:
### Dependent variable

In [99]:
y = data['class'].astype(int)

In [53]:
### Splitting the dataset

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.2)

In [102]:
X_train.shape

(19826, 12)

In [103]:
X_test.shape

(4957, 12)

In [104]:
y_train.shape

(19826,)

In [105]:
y_test.shape

(4957,)

In [59]:
### Model training and prediction

In [106]:
model = LogisticRegression()
model.fit(X_train,y_train)

C:\Users\Sasha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [107]:
y_pred = model.predict(X_test)
y_pred.shape

(4957,)

In [108]:
X_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11
13665,0.355,0.191,0.455,-0.4215,0.0,0.0,0.0,0.064673,-0.017150,-0.006064,-0.139469,0.036942
6754,0.239,0.000,0.761,-0.7845,0.0,1.0,0.0,-0.013348,0.192305,0.075076,-0.209852,-0.081329
6417,0.396,0.000,0.604,-0.8020,0.0,1.0,0.0,-0.034506,0.167953,0.130457,0.236565,-0.195023
10324,0.141,0.415,0.444,0.8126,0.0,0.0,0.0,0.224561,0.065046,0.237020,-0.229625,0.132010
4265,0.368,0.000,0.632,-0.5423,0.0,1.0,0.0,0.063315,0.206641,-0.056312,-0.026889,-0.041508


In [109]:
report = classification_report( y_test, y_pred )
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       285
           1       0.84      0.96      0.89      3815
           2       0.68      0.45      0.54       857

    accuracy                           0.82      4957
   macro avg       0.51      0.47      0.48      4957
weighted avg       0.76      0.82      0.78      4957



In [110]:
acc=accuracy_score(y_test,y_pred)
print("Logistic Regression with sentimenrtal analysis and TFIDF, Accuracy Score:" , acc)

Logistic Regression with sentimenrtal analysis and TFIDF, Accuracy Score: 0.8180351018761347


In [111]:
import pickle

In [112]:
pickle.dump(model, open('model.pkl', 'wb'))